<a href="https://colab.research.google.com/github/YusolCho/NLPstudy/blob/main/NLP_SentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **패키지 불러오기**

In [8]:
import pandas as pd 

## **데이터 불러오기**

In [9]:
from google.colab import drive
drive.mount('/content/drive')

review_df = pd.read_csv("/content/drive/My Drive/labeledTrainData.tsv", sep='\t', quoting=3)
review_df.head(3)


Mounted at /content/drive


,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."


1. id : 각 데이터의 id
2. sentiment : 영화평(review)의 Sentiment 결과 값(Target label)
* 1은 긍정적 평가
* 0은 부정적 평가
3. review : 영화평의 텍스트





In [10]:
print(review_df["review"][0]) #데이터의 첫번째 행에 해당되는 영화평의 텍스트 내용

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

## **노이즈 제거**
* HTML 형식에서 추출하여 태그 존재 -> 모두 공백으로 변경
* 영어가 아닌 특수문자/숫자 -> 모두 공백으로 변경(숫자도 Sentiment 를 위한 피처로 무의미!)





In [11]:
import re

# replace 함수 : <br> html 태그를 공백으로
review_df["review"] = review_df["review"].str.replace("<br />", " ")

# 정규표현식 모듈 re : 영어 문자열이 아닌 문자는 모두 공백으로 
review_df["review"] = review_df["review"].apply(lambda x : re.sub("[^a-zA-Z]", " ", x))


## **피처 데이터 셋 생성**

In [12]:
class_df = review_df["sentiment"]#sentiment(target변수)를 별도의 데이터셋으로 추출한 뒤
feature_df = review_df.drop(["id","sentiment"], axis = 1, inplace = False)#원본 데이터 셋에서 id와 sentiment 칼럼을 삭제

# ****이걸 왜하는지..?

## **데이터셋 분리(학습용, 테스트용)**

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(feature_df, class_df, test_size = 0.3, random_state = 156)

X_train.shape, X_test.shape

((17500, 1), (7500, 1))

## **벡터화하고 모델 돌리기 (1)ML(Logistic Regression) without TF-IDF**

## 패키지 불러오기

In [8]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer # CountVectorizer:빈도수에 따른 벡터화 TfidfVectorizer:TF-IDF. 
from sklearn.pipeline import Pipeline 
from sklearn.linear_model import LogisticRegression #Logistic Regression Model 구현 위함
from sklearn.metrics import accuracy_score, roc_auc_score #모델 평가 : accuracy, roc-curve, auc 구하기 위함 

* 여기서 sklearn.pipeline이란?
* -> 파이프라인은 데이터 전처리에서 모델학습까지의 과정을 하나로 연결해주는 것이라고 보면 된다.
* -> 파이프라인의 목적 : cross-validated될 수 있는 몇개의 단계를 다른 parameter 세팅이 가능하도록 조합하는 것. 이것을 위해 파이프라인에서 사용된 단계들 내에서 anova__k=10(anova의 parameter k의 값은 10)이런 식으로 
'이름 __파라미터=파라미터값' 라는 문법을 통해 파라미터 값 설정 가능하도록 한다.  
* https://zephyrus1111.tistory.com/254 여기 설명 참고 

* Cross-Validation(CV) 의 뜻
* -> 얘를 들어 가장 많이 사용되는 k-fold CV의 경우, training set를 k개로 쪼갠 다음 k-1개를 통해 모델을 학습시킨 뒤, 그 과정에서 찾은 최적의 parameter를 최종 parameter 로 채택하고 나머지 1개를 validation data로 사용하는 방법. 즉, 최적의 parameter를 찾기위한 기법 중 하나이다. 


## 파이프라인 구축 : 불용어 제거, 임베딩(using CountVectorization), 모델 구축

In [11]:
# 스톱 워드는 English, filtering, ngram은 (1,2)로 설정해 CountVectorizer 수행
# LogisticRegression의 C는 10으로 설정
pipeline = Pipeline([ # sklearn의 pipeline 구축. []로 묶여있는 건 steps로, 튜플형태
    ("cnt_vect", CountVectorizer(stop_words="english", ngram_range=(1,2))), #steps의 첫번째 튜플. Pipeline 의 첫 단계명은 cnt_vect
    ("lr_clf", LogisticRegression(C = 10,max_iter = 4000))]) #steps의 두번째 튜플. Pipeline 의 두번째 단계명은 lr_clf

## 모델 fitting

In [13]:
# pipeline 객체를 이용하여 fit(), predict()로 학습/예측 수행
# predict_proba()는 roc_auc 때문에 수행
pipeline.fit(X_train["review"], y_train)
pred = pipeline.predict(X_test["review"])
pred_probs = pipeline.predict_proba(X_test["review"])[:,1]

print("예측 정확도는 {0:.4f}, ROC-AUC는 {1:.4f}".format(accuracy_score(y_test, pred), roc_auc_score(y_test, pred_probs)))


예측 정확도는 0.8860, ROC-AUC는 0.9503


## **벡터화하고 모델 돌리기 (2)ML(Logistic Regression) with TF-IDF**

## 파이프라인 구축 : 불용어 제거, 임베딩(using TfidfVectorization), 모델 구축

In [14]:
# 스톱 워드는 English, filtering, ngram은 (1,2)로 설정해 CountVectorizer 수행
# LogisticRegression의 C는 10으로 설정
pipeline = Pipeline([
    ("cnt_vect", TfidfVectorizer(stop_words="english", ngram_range=(1,2))),
    ("lr_clf", LogisticRegression(C = 10))])

## 모델 fitting

In [18]:
print(pipeline.score(X_train["review"],y_train))

0.9998285714285714


In [16]:
# pipeline 객체를 이용하여 fit(), predict()로 학습/예측 수행
# predict_proba()는 roc_auc 때문에 수행
pipeline.fit(X_train["review"], y_train)
pred = pipeline.predict(X_test["review"])
pred_probs = pipeline.predict_proba(X_test["review"])[:,1]

print("예측 정확도는 {0:.4f}, ROC-AUC는 {1:.4f}".format(accuracy_score(y_test, pred), roc_auc_score(y_test, pred_probs)))


예측 정확도는 0.8936, ROC-AUC는 0.9598


## **벡터화하고 모델 돌리기 (3)ML > Lexicon(비지도학습) > SentiWordNet**

* Lexicon?
* 일종의 감성 어휘 사전이라고 생각하면 됨 
* 감성 분석을 위한 용어, 문맥에 대한 다양한 정보 보유 -> 문서의 긍부정 판단


[SentiWordNet을 이용한 감성 분석]

1. 문장 토큰화
2. 다시 문장을 단어토큰화 + 품사 태깅
3. 품사 태깅된 단어 기반으로 synset 객체와 senti_synset 객체를 생성
4. Senti_synset에서 긍정 감성 / 부정 감성 지수를 구하고 이를 모두 합산하여 특정 임계치 값 이상일 때 긍정 감성으로, 그렇지 않을 때는 부정 감성으로 결정

* 여기서 문득 궁금한 점 : 품사태깅 해주는 이유?
* 품사태깅은 POS(Part of Speech) Tagging으로, 텍스트를 형태소 단위로 쪼개준 뒤 해당 형태소의 품사를 태깅하는 것을 말함 
* 품사태깅의 활용 : 품사 태깅을 적용한 후, 명사만 추출하거나 주요 품사만 추출해 데이터로 사용할 수 있다. 예를 들어 영화 리뷰의 긍/부정을 파악하는데 '이다'라는 단어는 분류에 직접적으로 영향을 미치지 않을 것이라는 가정하에 주요 품사만 추출해 데이터로 사용한다.


## 패키지 임포트 

In [1]:
# 품사 태깅하는 내부 함수

from nltk.corpus import wordnet as wn
# ****************wordnet이 뭔지 잘 모름. 

## 품사 태깅

In [2]:

# 간단한 NLTK PennTreebank Tag를 기반으로 WordNet 기반의 품사 Tag로 변환
def penn_to_wn(tag):
  if tag.startswith("J"):
    return wn.ADJ
  elif tag.startswith("N"):
    return wn.NOUN
  elif tag.startswith("R"):
    return wn.ADV
  elif tag.startswith("V"):
    return wn.VERB

## 패키지 추가 임포트

In [3]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag


## 감성지수 구하는 함수 정의하기...?
******어려워...

In [4]:
def swn_polarity(text):
  # 감성 지수 초기화
  sentiment = 0.0
  tokens_count = 0

  lemmatizer = WordNetLemmatizer()
  raw_sentences = sent_tokenize(text)
  # 분해된 문장별로 단어 토근 --> 품사 태깅 후에 SentiSynset 생성 --> 감성 지수 합산
  for raw_sentences in raw_sentences :
    # NLTK 기반의 품사 태깅 문장 추출
    tagged_sentence = pos_tag(word_tokenize(raw_sentences))
    for word, tag in tagged_sentence:

      # WordNet 기반 품사 태깅과 어근 추출
      wn_tag = penn_to_wn(tag)
      if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV):
        continue
      lemma = lemmatizer.lemmatize(word, pos = wn_tag)
      
      if not lemma:
        continue
      
      # 어근을 추출한 단어와 WordNet 기반 품사 태깅을 입력해 Synset 객체 생성
      synsets = wn.synsets(lemma, pos = wn_tag)
      if not synsets:
        continue
      
      # sentiwordnet의 감성 단어 분석으로 감성 synset 추출
      # 모든 단어에 대해 긍정 감성 지수는 +로, 부정 감성 지수는 -로 합산하여 감성 지수 계산
      synset = synsets[0]
      swn_synset = swn.senti_synset(synset.name())
      sentiment += (swn_synset.pos_score() - swn_synset.neg_score())
      tokens_count += 1

  if not tokens_count :
    return 0

  # 총 score가 0 이상일 경우 긍정(Positive) 1, 그렇지 않을 경우 부정(Negative) 0 반환
  if sentiment >= 0:
    return 1

  return 0

## ? 뭐시기 다운로드

In [6]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('sentiwordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package sentiwordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/sentiwordnet.zip.


True

In [15]:
review_df["preds"] = review_df["review"].apply( lambda x : swn_polarity(x))
y_target = review_df["sentiment"].values
preds = review_df["preds"].values

KeyboardInterrupt: ignored

## 성능 평가 (분류 성능평가)

In [16]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

def get_clf_eval(y_test, pred):
  confusion = confusion_matrix(y_test, pred)
  accuracy = accuracy_score(y_test, pred)
  precision = precision_score(y_test, pred)
  recall = recall_score(y_test, pred)
  print("오차행렬")
  print(confusion)
  print("정확도 : {0:.4f}, 정밀도 : {0:.4f}, 재현율 : {0:.4f}". format(accuracy, precision, recall))

In [17]:
print(" #### SentiWordNet 예측 성능 평가")
get_clf_eval(y_target, preds)

 #### SentiWordNet 예측 성능 평가
오차행렬
[[7668 4832]
 [3636 8864]]
정확도 : 0.6613, 정밀도 : 0.6613, 재현율 : 0.6613


## **벡터화하고 모델 돌리기 (4)ML > Lexicon(비지도학습) > VADER Lexicon**
* VADER는 소셜 미디어의 감성 분석 용도로 만들어진 룰 기반의 Lexicon

[Vader를 이용한 감성 분석]

(1)먼저 SentimentIntensityAnalyzer 객체를 생성

(2)문서별로 polarity_scores() 메서드를 호출해 감성 점수를 구하기

(3)해당 문서의 감성 점수가 특정 임계값 이상이면 긍정, 그렇지 않으면 부정으로 판단

SentimentIntensityAnalyzer 객체의 polarity_scores() 메서드
: 딕셔너리 형태의 감성 점수를 반환.

* "neg" : 부정 감성 지수
* "neu" : 중립적인 감성 지수
* "pos" : 긍정 감성 지수
* compound : 감성 지수. neg, neu, pos score를 적절히 조합해 -1에서 1 사이의 값을 가진다. 이 compound score를 기반으로 부정 감성 또는 긍정 감성 여부를 결정
* 임계값 : 보통 0.1 이상이면 긍정 감성, 그 이하이면 부정 감성으로 판단(케바케. 임계값 조절가능)

# 뭐시기 다운로드, 패키지 임포트

In [18]:
import nltk
nltk.download('vader_lexicon')

from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


## SentimentIntensityAnalyzer 객체를 생성

In [19]:
senti_analyzer = SentimentIntensityAnalyzer()
senti_scores = senti_analyzer.polarity_scores(review_df["review"][0]) #polarity_scores() 메서드 : 딕셔너리 형태의 감성 점수를 반환.
print(senti_scores)

{'neg': 0.13, 'neu': 0.743, 'pos': 0.127, 'compound': -0.7943}


## 이 과정이 왜 필요한건지 모르겠음 그냥 위에 함수처럼 해주면 되는거 아닌가 싶고.. 


In [21]:
def vader_polarity(review, threshold = 0.1): #threshold로 임계값을 설정한다. 
  analyzer = SentimentIntensityAnalyzer() #객체 생성
  scores = analyzer.polarity_scores(review) #SentimentIntensityAnalyzer()의 객체의 polarity_scores() 메서드 : 딕셔너리 형태의 감성 점수를 반환.

  # compound 값에 기반해 threshold 입력값보다 크면 1, 그렇지 않으면 0을 반환
  agg_score = scores["compound"] #감성 점수가 할당되어있는 scores 변수 내의 compound 값을 추출
  final_sentiment = 1 if agg_score >= threshold else 0 #그 compound 값을 threshold 값과 비교하여 0또는 1 할당
  return final_sentiment

## 

In [22]:
# apply lambda 식을 이용해 레코드별로 vader_polarity를 수행하고 결과를 "vader_preds"에 저장
review_df["vader_preds"] = review_df["review"].apply( lambda x : vader_polarity(x, 0.1))
#문서에 새로운 열을 추가하고, 문서의 각 행에 대해 위의 vader_polarity함수를 적용한 결과값을 그 열에 담는다 
y_target = review_df["sentiment"].values #실제값
vader_preds = review_df["vader_preds"].values #예측값 

print("#### VADER 예측 성능 평가 ####")
get_clf_eval(y_target, vader_preds)

#### VADER 예측 성능 평가 ####
오차행렬
[[ 6747  5753]
 [ 1858 10642]]
정확도 : 0.6956, 정밀도 : 0.6956, 재현율 : 0.6956


* *********apply(lambda x: 이거 처음보는건데 뭐임